In [1]:
import numpy as np
import bokeh as bk
from bokeh.io import push_notebook
import bokeh.plotting as bkp
import bokeh.models as bkm
from bokeh.io import push_notebook
from ipywidgets import interact
import ipywidgets as widgets
bkp.output_notebook()

Loading BokehJS ...

# Solidification week 5 homework

## Mikko Karkkainen 11740378

### Problem 1. 

Consider a carbon steel casting (0.8%C) of dimensions 37mm height x 33mm x 33mm. Calculate the cooling rate (G•V)for this casting. Demonstrate that the microstructure of this casting cannot be planar. What microstructure will you expect? Justify your answer.

### Solution:

Average solidification velocity is given by 

$$ V = (v/A)/t_f$$

The final solidification time can be obtained from the Chvorinov equation, presuming a low Biot number (i.e. presuming liquid has uniform temperature). Time-stepping Chvorinov:

$$ T^{n+1} = T^{n} - \frac{A}{v \rho} \frac{T^n-T_m}{\Delta H_f/\Delta T_0 + c} \sqrt{\frac{k_m \rho_m c_m}{\pi t^{n+1}}} \Delta t $$

Let's presume a silica sand mold, to have a minimum thermal gradient.

Data taken for AISI 1008 Carbon steel from Nova database:

<table>
    <tr>
        <th>Material property</th>
        <td>$ \rho $</td>
        <td> $ c $ </td>
        <td> $ k $ </td>
        <td> $ v $</td>
        <td> $ A $</td>
        <td> $ \Delta H_F $ </td>
        <td> $  T_L $ </td>
        <td> $  T_S $ </td>
        <td> $ \Delta T_0 $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td> 7200 </td>
        <td> 687 </td>
        <td> 30 </td>
        <td> 4.029×10^-5 m^3 </td>
        <td> 0.001089 m^2 </td>
        <td> 270 000 </td>
        <td> 1800.241 K </td>
        <td> 1772.705 K </td>
        <td> 27.54 K </td>
    </tr>
</table>

Mold properties:

<table>
    <tr>
        <th>Material property</th>
        <td> $ \rho_m $ </td>
        <td> $ k_m $ </td>
        <td> $ c_m $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td>1600</td>
        <td>0.52</td>
        <td>1170</td>
    </tr>
</table>    

Calculate The solidification time:

In [2]:
Delta_t=1
T_i=1800.241
Delta_T_0=27.54
v=4.029*10**(-5)
A=0.001089
Delta_H_f=270000
rho=7200
c=687
T_L=1800.241
T_S=1772.705

#silica sand mold:

rho_si=1600
k_si=0.52
c_si=1170


#simulation parameters:

step_count=2230

temp_values_si=np.empty(step_count)
temp_values_si[0]=T_i
    
def calc_temp(rho_m,k_m,c_m,temp_values):
    for i in range(1,step_count):
        if (temp_values[i-1]>T_L or temp_values[i-1]<=T_S):
            temp_values[i]=temp_values[i-1]-A/v*(temp_values[i-1]-298.15)/(rho*(c))*np.sqrt((k_m*rho_m*c_m/(np.pi*(Delta_t*i))))*Delta_t
            
        else:
            temp_values[i]=temp_values[i-1]-A/v*(temp_values[i-1]-298.15)/(rho*(Delta_H_f/Delta_T_0 + c))*np.sqrt((k_m*rho_m*c_m/(np.pi*(Delta_t*i))))*Delta_t

calc_temp(rho_si,k_si,c_si,temp_values_si)
        
f=bkp.figure(title="Cooling curve of Fe-0.8%C casting", x_axis_label="time (s)", y_axis_label="Temperature (K)")
time_series=np.linspace(0,step_count*Delta_t,step_count)
time_series_h=np.linspace(Delta_t,step_count*Delta_t,step_count)

f.line(time_series,temp_values_si, color="red", legend="Silica sand mold")

bkp.show(f)

The total solidification time is approximately 2230 seconds. Therefore:

$$ V = (v/A)/t_f = 0.037 m / 2230 s = 1.66 \cdot 10^{-5} m/s $$


Thermal gradient equation from book:

$$ G_T =  (\rho_s \Delta H_f V)/k_s = 7200 \cdot 270000 \cdot 1.66 \cdot 10^{-5} / 30 = 1075.68 K/m  $$

Therefore:
    
$$ \frac{G_T}{V} =  \frac{1075.68}{1.66×10^-5} = 6.48 \cdot 10^7 Ks/m^2 $$

    


Non-steady state criterion for constitutional undercooling:

$$ \frac{G_T}{V} < - \frac{m_L C_L^* (1-k)}{D_L}$$

At the start of solidification, $ C_L^* = C_0 $

Required values:
    
<table>
    <tr>
        <th>Material property</th>
        <td> $ m_L $ </td>
        <td> $ C_0 $ </td>
        <td> $ k $ </td>
        <td> $ D_L $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td>$ -61 $</td>
        <td>$ 0.8 $</td>
        <td>$ 0.28 $</td>
        <td style="width:70px">$ 2\cdot10^{-8} $</td>
    </tr>
</table>   
    

$$ - \frac{m_L C_0 (1-k)}{D_L} = - \frac{-61 \cdot 0.8 (1-0.28)}{2\cdot10^{-8}} = 1.7568 \cdot 10^9 $$

$$ \frac{1.7568 \cdot 10^9}{6.48 \cdot 10^7} \approx 27.1$$

<img src="https://dev-plebbit.s3.amazonaws.com/uploads/topic/picture/21/planar_solidification.png"/>

The ratio $ \frac{G_T}{V} $ required for planar solidification is approximately 27.1 times greater than the one simulated for the casting. This assumes however, that there is no interfacial energy required for solidification, or any temperature gradient within the solid. Without these assumptions, the requirement is not as high.

### Problem 2. 

Consider the graph in Fig. 8.16 in the textbook. According to the experimental data in the graph, the coarsening constant is $10^{-16} m/s^3$. Using the equations in Table 8.3, find out which assumption gives results closer to the experimental data. Provide a table with all the data used in your calculations. Then, using the best equation, calculate the coarsening constant for an Al-3% Cu and Fe-0.3% C alloys. Comment on the results.

Below is shown figure 8.16 (after Fleming 1991)  for Al-4.5% Cu alloys

<img src="https://dev-plebbit.s3.amazonaws.com/uploads/topic/picture/19/fleming_1991.png"/>

Below is shown table 8.3

<img src="https://dev-plebbit.s3.amazonaws.com/uploads/topic/picture/20/Coarsening_constants.png"/>

To solve this problem we need to calculate the values for coarsening constant and see which gives the result that is closest to $10^{-16} m/s^3$

Required material properties for Al-4.5 Cu from Appendix B

<table>
    <tr>
        <th>Material property</th>
        <td> $ m_L $ </td>
        <td> $ C_0 $ </td>
        <td> $ k $ </td>
        <td> $ D_L $ </td>
        <td> $ \Gamma $ </td>
        <td> $ C_E $ </td>
        <td> $ \gamma_{SL} $ </td>
        <td> $ v^2_m $ </td>
        <td> $ T_L $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td>$ -3.6 $</td>
        <td>$ 4.5 $</td>
        <td>$ 0.14 $</td>
        <td style="width:80px">$ 2.8 \cdot10^{-9} $</td>
        <td style="width:80px">$ 2.4 \cdot10^{-7} $</td>
        <td>$ 32.7 $</td>
        <td>$ 0.093 $</td>
        <td style="width:80px">$ 11 \cdot10^{-6} $</td>
        <td> $ 923.15 $ </td>
        
    </tr>
</table>   

Let us estimate the liquid concentration $ C_L $ as a function of solid fraction $ f_s $ using the scheil function. Then let's plot the coarsening constants as a function of solid fraction.

In [38]:


D_L=2.8*10**(-9)
Gamma=2.4*10**(-7)
C_E=32.7
C_0=4.5
m_L=-3.6
gamma_SL=0.093
v2m=11*10**(-6)
k=0.14
R=8.314
T_L=923.15
T_S=821.15

def Temp(x):
    return 923.15+x*(821.15-923.15)/(1-0)
    
def C_L_Scheil(x):
    return C_0*(1-x)**(k-1)


def Kattamis_Flemings(x):
    return (20*D_L*Gamma*np.log(C_E/C_0))/(m_L*C_L_Scheil(x)*(k-1)*(C_E-C_0))

def Ardell(x):
    return (v2m*gamma_SL*C_0*D_L)/(R*Temp(x))*(1-C_0/100)*(1-x)/x

def Mortensen_s(x):
    return (27*D_L*Gamma)/(2*m_L*C_L_Scheil(x)*(k-1)*x**(2/3)*(1-x**(1/3)))

def Mortensen_c(x):
    return (27*D_L*Gamma)/(4*m_L*C_L_Scheil(x)*(k-1)*x**(1-x**(1/2)))


In [39]:


f=bkp.figure(title="coarsening constant, function of fs", x_axis_label="f_s", y_axis_label="mu_0",y_axis_type="log", y_range=(10**(-18), 10**(-14)))

x_array=np.linspace(0.01,0.99,100)

f.line(x_array,Kattamis_Flemings(x_array), color="red", legend="Kattamis_Flemings")

f.line(x_array,Ardell(x_array), color="blue", legend="Ardell")

f.line(x_array,Mortensen_s(x_array), color="yellow", legend="Mortensen sphere")

f.line(x_array,Mortensen_c(x_array), color="green", legend="Mortensen cylinder")

bkp.show(f)

From $ f_S $ 0.12 to 0.99, Mortensen cylinder is within an order of magnitude of $ 10^{-16} $, so it is the best fit.

Let's get the material data for Al-3% Cu and Fe - 0.3% C:

### Al-3% Cu
<table>
    <tr>
        <th>Material property</th>
        <td> $ m_L $ </td>
        <td> $ C_0 $ </td>
        <td> $ k $ </td>
        <td> $ D_L $ </td>
        <td> $ \Gamma $ </td>
        <td> $ C_E $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td>$ ? $</td>
        <td>$ 3 $</td>
        <td>$ ? $</td>
        <td style="width:80px">$ ? $</td>
        <td style="width:80px">$ ? $</td>
        <td>$ ? $</td>
    
    </tr>
</table>

### Fe-0.3 % C

<table>
    <tr>
        <th>Material property</th>
        <td> $ m_L $ </td>
        <td> $ C_0 $ </td>
        <td> $ k $ </td>
        <td> $ D_L $ </td>
        <td> $ \Gamma $ </td>
        <td> $ C_E $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td>$ ? $</td>
        <td>$ 0.3 $</td>
        <td>$ ? $</td>
        <td style="width:80px">$ ? $</td>
        <td style="width:80px">$ ? $</td>
        <td>$ ? $</td>
        
    </tr>
</table>

Let's calculate the coarsening constant for these two materials:

In [31]:
D_L_1=2.8*10**(-9)
Gamma_1=2.4*10**(-7)
C_E_1=32.7
C_0_1=4.5
m_L_1=-3.6
k_1=0.14

D_L_2=2.8*10**(-9)
Gamma_2=2.4*10**(-7)
C_E_2=32.7
C_0_2=4.5
m_L_2=-3.6
k_2=0.14

def C_L_Scheil(x,k):
    return C_0*(1-x)**(k-1)

def Mortensen_c(x,Gamma,D_L,m_L,k):
    return (27*D_L*Gamma)/(4*m_L*C_L_Scheil(x,k)*(k-1)*x**(1-x**(1/2)))

### Problem 3. 

Calculate the change in secondary arm spacing for a Fe-0.9%C alloy for local solidification times between $10^{-2}$ and $ 10^3 $ seconds (plot a λ - tf graph). Note:Assume $ C_L=(C_E-C_0)/2$

### Solution:

We can calculate the secondary dendrite arm spacing using the equation:

$ \lambda_2 = \mu_0^{1/3} \cdot t_f^{1/3} $

First let's calculate the value of the coarsening coefficient using Mortensen's dynamic coarsening for cylinders, assuming that $ CL=(CE−C0)/2 $ and that $ f_s=0.5 $

### Fe-0.9% C

<table>
    <tr>
        <th>Material property</th>
        <td> $ m_L $ </td>
        <td> $ C_0 $ </td>
        <td> $ k $ </td>
        <td> $ D_L $ </td>
        <td> $ \Gamma $ </td>
        <td> $ C_E $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td>$ ? $</td>
        <td>$ 0.9 $</td>
        <td>$ ? $</td>
        <td style="width:80px">$ ? $</td>
        <td style="width:80px">$ ? $</td>
        <td>$ ? $</td>
        
    </tr>
</table>

In [34]:
Gamma_3=1
D_L_3=0
m_L_3=0
k_3=0

mu= Mortensen_c(0.5,Gamma_3,D_L_3,m_L_3,k_3)

ZeroDivisionError: float division by zero

In [36]:
def lambda_2(x):
    return mu**(1/3)*x**(1/3)

In [37]:
f=bkp.figure(title="coarsening constant, function of fs", x_axis_label="f_s", y_axis_label="mu_0",y_axis_type="log", x_axis_type="log")

t_array=np.linspace(10**(-2),10**3,100)

f.line(t_array,lambda_2(t_array), color="green", legend="Mortensen cylinder")

bkp.show(f)

NameError: name 'mu' is not defined

### Problem 4.

Consider a micro-volume element of an Al-4.5%Cu alloy cooled at constant heat extraction rate of $ \dot{Q}=2\cdot 10^8 Jm^{-3}s^{-1} $, from an initial temperature of 700°C. The volumetric grain density is of 100 grains/mm3. Calculate and graph:

a. the temperature and solid fraction evolution during the solidification

b. the evolution of the liquid composition as a function of the fraction solid

c. the evolution of the SDAS as a function of fraction solid, using Mortensen’s model for dynamic coarsening of spheres. Compare your result with the graph in Fig. 8.16 textbook.

Note : use as an example application 8.5 textbook

### Solution

The governing heat transfer equation:

$$ \dot Q = \rho \Delta H_f \frac{\partial fs}{\partial t} - \rho c \frac{\partial T}{\partial t}  $$

Discretized form:

$$ T^{new} = T^{old} - \frac{\dot Q}{\rho c}\Delta t + \frac{\Delta H_f}{c} \Delta f_S^{new} $$

### Al-Cu 4.5 %

<table>
    <tr>
        <th>Material property</th>
        <td> $ m_L $ </td>
        <td> $ C_0 $ </td>
        <td> $ k $ </td>
        <td> $ D_L $ </td>
        <td> $ \Gamma $ </td>
        <td> $ C_E $ </td>
        <td> $ \gamma_{SL} $ </td>
        <td> $ v^2_m $ </td>
        <td> $ T_L $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td>$ -3.6 $</td>
        <td>$ 4.5 $</td>
        <td>$ 0.14 $</td>
        <td style="width:80px">$ 2.8 \cdot10^{-9} $</td>
        <td style="width:80px">$ 2.4 \cdot10^{-7} $</td>
        <td>$ 32.7 $</td>
        <td>$ 0.093 $</td>
        <td style="width:80px">$ 11 \cdot10^{-6} $</td>
        <td> $ 923.15 $ </td>
        
    </tr>
</table>  

In [91]:
q=2*10**(8)
temp_init=923.15
delta_t=0.01
Gamma=2.4*10**(-7)
C_0=4
D_L=5.4*10**(-9)
k=0.14
N=100
rho=2325.87
c=1170
delta_H_f=3.91*10**5
T_f=923.15
m=-14


def C_L_new(fs_old):
    #print(C_0*(1-fs_old)**(k-1))
    return C_0*(1-fs_old)**(k-1)

def Delta_T(T_f,T_old,fs_old):
    #print(T_f+m*C_L_new(fs_old)-T_old)
    if T_f+m*C_L_new(fs_old)-T_old >= 0:
        return T_f+m*C_L_new(fs_old)-T_old
    else:
        return 0

def V_S(T_f,T_old,fs_old):
    #print(D_L/(2*np.pi**2*Gamma*m*(k-1)*C_L_new(fs_old))*(Delta_T(T_f,T_old,fs_old))**2)
    return D_L/(2*np.pi**2*Gamma*m*(k-1)*C_L_new(fs_old))*(Delta_T(T_f,T_old,fs_old))**2

def R_new(r_old,T_f,T_old,fs_old):
    #print(r_old+V_S(T_f,T_old,fs_old)*delta_t)
    return r_old+V_S(T_f,T_old,fs_old)*delta_t

def Delta_fs(r_old,T_f,T_old,fs_old):
    #print(4*np.pi*(R_new(r_old,T_f,T_old,fs_old))**(2)*V_S(T_f,T_old,fs_old)*delta_t*N)
    return 4*np.pi*(R_new(r_old,T_f,T_old,fs_old))**(2)*V_S(T_f,T_old,fs_old)*delta_t*N
    
def T_new(r_old,T_f,T_old,fs_old):
    #print(T_old-q/(rho*c)*delta_t+delta_H_f/c*Delta_fs(r_old,T_f,T_old,fs_old))
    return T_old-q/(rho*c)*delta_t+delta_H_f/c*Delta_fs(r_old,T_f,T_old,fs_old)

def fs_new(r_old,T_f,T_old,fs_old):
    #print(fs_old+Delta_fs(r_old,T_f,T_old,fs_old))
    return fs_old+Delta_fs(r_old,T_f,T_old,fs_old)

max_steps=1000
time=np.linspace(0,1000,max_steps)
fs_arr=np.zeros(max_steps)

T_arr=np.zeros(max_steps)
T_arr[0]=temp_init

r_arr=np.zeros(max_steps)
r_arr[0]=10**(-7)


for i in range(1,len(time)):
    T_arr[i]=T_new(r_arr[i-1],T_f,T_arr[i-1],fs_arr[i-1])
    fs_arr[i]=fs_new(r_arr[i-1],T_f,T_arr[i-1],fs_arr[i-1])
    r_arr[i]=R_new(r_arr[i-1],T_f,T_arr[i-1],fs_arr[i-1])
    

In [92]:
f=bkp.figure(title="Evolution of T during dendrite growth", x_axis_label="Time,s", y_axis_label="Temperature, C")

f.line(time*delta_t,T_arr, color="red", legend="Temperature")

bkp.show(f)


In [93]:
f=bkp.figure(title="Evolution of fs dendrite growth", x_axis_label="Time,s", y_axis_label="fs")


f.line(time*delta_t,fs_arr, color="red", legend="fraction of solid")

bkp.show(f)

Now lets calculate the SDAS using mortensen's equation for spheres.

In [102]:
f=bkp.figure(title="Evolution of fs dendrite growth", x_axis_label="fs", y_axis_label="SDAS")

def lambda_2(x):
    return Mortensen_s(x)**(1/3)*x**(1/3)

f.line(fs_arr,lambda_2(fs_arr), color="yellow", legend="Mortensen sphere")

bkp.show(f)

### Problem 5.

Consider the binary Al-Si alloy phase diagram, making the following assumptions for the eutectic invariant and liquidus lines (compositions in wt%, temperatures in °C)

$$ T_{liq}^{Al} = 577-6.8(C_l - 12.2) \;\; ; \;\; T_{liq}^{Si} = 577 +9.53( C_l -12.2)  $$

$$ T_{eut} = 577 \;\; ; \;\; C_{eut} = 12.2$$

Compute the skewed coupled zone of Al-Si for a thermal gradient of $ G=10 K/mm $ and $D_l = 3\cdot 10^{-3} mm^2s^{-1} $. Compare the results with the microstructures shown in Fig. 9.35